In [1]:
%%capture
!python3 -m pip install pandas numpy matplotlib tensorflow tqdm

In [2]:
import requests
import urllib3
from urllib.parse import urlparse
import socket

import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

In [3]:
import warnings
warnings.filterwarnings(action="ignore", module="urllib3")

In [4]:
raw_data = pd.read_csv("./raw_data.csv")
raw_data

,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement
...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing
651188,www.gamespot.com/xbox360/action/deadspace/,phishing
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing


In [5]:
# def generate_alternative_urls(data_in):
#     original_url = data_in
#     url_with_scheme = "http://"+original_url if original_url[0:4] != "http" else original_url
#     parsed_url = urlparse(url_with_scheme)
#     scheme = parsed_url.scheme
#     host = parsed_url.netloc
#     path = parsed_url.path
#     query = parsed_url.query

#     a = np.empty((8, 8))
#     a.fill(np.nan)
#     alternative_urls = pd.DataFrame(a, columns=["url", "type", "content", "http", "path", "query"])
#     print(alternative_urls)
    
#     # Encoding: [ (scheme == "http"), (host != ""), (path != ""), (query != "") ]
    
#     alternative_urls[0,""] = "http://"+host
#     if(path != ""):
#         alternative_urls[0, 1, 1, 0] = "http://"+host+path
#     if(query != ""):
#         alternative_urls[0, 1, 0, 1] = "http://"+host+"?"+query
#     if(path != "" and query != ""):
#         alternative_urls[0, 1, 1, 1] = "http://"+host+path+"?"+query
        
#     alternative_urls[1, 1, 0, 0] = "https://"+host
#     if(path != ""):
#         alternative_urls[1, 1, 1, 0] = "https://"+host+path
#     if(query != ""):
#         alternative_urls[1, 1, 0, 1] = "https://"+host+"?"+query
#     if(path != "" and query != ""):
#         alternative_urls[1, 1, 1, 1] = "https://"+host+path+"?"+query
        
#     print(alternative_urls)
#     return alternative_urls

# #generate_alternative_urls(raw_data.loc[0, "url"])
# #raw_data.loc[:20, "url"].apply(generate_alternative_urls)
# generate_alternative_urls("http://test1.com/test2?test3")

In [6]:
MAX_QUERIES = 100
TIMEOUT = (5, 5)

In [7]:
def get_location_data(original_url):
    url_with_scheme = "http://"+original_url if original_url[0:4] != "http" else original_url
    parsed_url = urlparse(url_with_scheme)
    host = parsed_url.netloc
    ip_addr = socket.gethostbyname(host)
    json = requests.get("https://ipinfo.io/"+ip_addr+"/json").json()
    return json["city"], json["region"], json["country"]
    

In [8]:
def download_data(start=0, end=MAX_QUERIES, timeout=TIMEOUT):
    for row_index in tqdm(range(start, end if len(raw_data["url"]) > end else len(raw_data["url"]))):
            original_url = raw_data.loc[row_index, "url"]
            try:
                r = requests.get(original_url, verify=False, timeout=timeout)
                raw_data.loc[row_index, "content"] = r.content
                raw_data.loc[row_index, "status_code"] = r.status_code
                
                redirects = 0
                for r_history in r.history:
                    if(r_history.status_code == 301):
                        redirects = redirects + 1
                raw_data.loc[row_index, "redirects"] = redirects

                city, region, country = get_location_data(original_url)
                raw_data.loc[row_index, "city"] = city
                raw_data.loc[row_index, "region"] = region
                raw_data.loc[row_index, "country"] = country
            except socket.error:
                pass
    raw_data.iloc[start:end].to_csv("./data.csv")
    return raw_data.iloc[start:end]

In [9]:
try:
    data = pd.read_csv("./data.csv", index_col=0)
    if(data.shape[0] != MAX_QUERIES):
        data = download_data()
        
except FileNotFoundError:
    data = download_data()
data = data.dropna()
data = data.reset_index(drop=True)
data

  0%|          | 0/100 [00:00<?, ?it/s]

,url,type,content,status_code,redirects,city,region,country
0,http://www.szabadmunkaero.hu/cimoldal.html?sta...,defacement,"b'<?xml version=""1.0"" encoding=""iso-8859-1""?>\...",404.0,0.0,Budapest,Budapest,HU
1,http://larcadelcarnevale.com/catalogo/palloncini,defacement,"b'<!DOCTYPE html>\n<html data-adblockkey=""MFww...",200.0,0.0,Munich,Bavaria,DE
2,http://hollywoodlife.com/2014/05/01/rihanna-ih...,benign,"b'<!DOCTYPE html>\n<html lang=""en-US"">\n\t<hea...",404.0,1.0,San Francisco,California,US
3,http://peluqueriadeautor.com/index.php?option=...,defacement,"b'<!DOCTYPE HTML PUBLIC ""-//IETF//DTD HTML 2.0...",404.0,0.0,Madrid,Madrid,ES
4,http://www.824555.com/app/member/SportOption.p...,malware,"b'<html><head></head><body><a href="""" id=""hao1...",200.0,0.0,Los Angeles,California,US
5,http://www.raci.it/component/user/reset.html,defacement,b' <!DOCTYPE html>\...,404.0,1.0,Turin,Piedmont,IT
6,https://docs.google.com/spreadsheet/viewform?f...,phishing,"b'<!DOCTYPE html><html lang=""en"" class=""HB1eCd...",200.0,2.0,Washington,"Washington, D.C.",US
7,http://earth-sure.com/industrial-tech/everlast...,defacement,b'<head><title>Not Acceptable!</title></head><...,406.0,0.0,Provo,Utah,US
8,http://interpark.com/displaycorner/FreeMarket....,benign,"b'\r\n\r\n<!DOCTYPE HTML PUBLIC ""-//W3C//DTD H...",200.0,0.0,Seoul,Seoul,KR
9,http://www.kingsmillshotel.com/spring/mothers-day,defacement,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,200.0,3.0,Manchester,England,GB


In [10]:
data[data["status_code"] == 200]

,url,type,content,status_code,redirects,city,region,country
1,http://larcadelcarnevale.com/catalogo/palloncini,defacement,"b'<!DOCTYPE html>\n<html data-adblockkey=""MFww...",200.0,0.0,Munich,Bavaria,DE
4,http://www.824555.com/app/member/SportOption.p...,malware,"b'<html><head></head><body><a href="""" id=""hao1...",200.0,0.0,Los Angeles,California,US
6,https://docs.google.com/spreadsheet/viewform?f...,phishing,"b'<!DOCTYPE html><html lang=""en"" class=""HB1eCd...",200.0,2.0,Washington,"Washington, D.C.",US
8,http://interpark.com/displaycorner/FreeMarket....,benign,"b'\r\n\r\n<!DOCTYPE HTML PUBLIC ""-//W3C//DTD H...",200.0,0.0,Seoul,Seoul,KR
9,http://www.kingsmillshotel.com/spring/mothers-day,defacement,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,200.0,3.0,Manchester,England,GB
13,http://www.pn-wuppertal.de/links/2-linkseite/5...,defacement,"b'\xef\xbb\xbf\n<!DOCTYPE html PUBLIC ""-//W3C/...",200.0,1.0,Karlsruhe,Baden-Wurttemberg,DE
16,http://www.parafiapiaski.pl/index.php?option=c...,defacement,b'<!doctype html>\n<html>\n\n<head>\n\n<meta h...,200.0,0.0,Warsaw,Mazovia,PL
17,http://fanpage.gr/family/gineka/%ce%b1%cf%85%c...,benign,"b'<!DOCTYPE html>\n<html lang=""el"">\n<head>\n<...",200.0,3.0,San Francisco,California,US
18,http://worldoftanks.ru/ru/content/guide/paymen...,benign,"b'<!DOCTYPE html>\n<html lang=""ru"">\n <head...",200.0,1.0,Luxembourg,Luxembourg,LU
21,http://www.viamanaus.com.br/index.php?option=c...,defacement,b'\r\n\r\n\r\n <!DOCTYPE html>\r\n <...,200.0,1.0,São Paulo,São Paulo,BR


## TODO
1. Run this on the entire data set
2. Do one-hot-encoding for the city, region, country, and type (probably using tensorflow/keras/sklearn)
3. Do text encoding on url and content (probably using tensorflow/keras/sklearn)
4. Split training and testing (using tensorflow/keras/sklearn)
5. Implement Z-Fold cross-validation (probably using tensorflow/keras/sklearn)
6. Train CNN using tensorflow/keras/sklearn
7. Optinally, fine tune hyperparameters (e.g. epochs, activation function, layers, etc.)
8. Graph results